In [9]:
# !pip install icypher
# !pip install neo4j
# !pip install yfiles_jupyter_graphs


In [10]:
from neo4j import GraphDatabase
import time
import ipywidgets as widgets
from IPython.display import display
URI = "bolt://localhost:7687"
AUTH = ("neo4j", "neo4j123")

# Create Constraints

In [11]:
airport_constraint = """
CREATE CONSTRAINT airport_unique IF NOT EXISTS
FOR (a:Airport)
REQUIRE a.iata IS UNIQUE;
"""

airline_constraint = """
CREATE CONSTRAINT airline_unique IF NOT EXISTS
FOR (a:Airline)
REQUIRE a.iata IS UNIQUE;
"""

route_constraint = """
CREATE CONSTRAINT route_node_key IF NOT EXISTS
FOR (r:Route)
REQUIRE (r.departure, r.arrival) IS NODE KEY;
"""

# Load Airport Nodes

In [12]:

query_airports = """
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/Cryogenic117/graph-summit-demo/refs/heads/main/df_airports.csv" AS row
WITH
  trim(row.Airport_IATA) AS iata,
  trim(row.City) AS city,
  trim(row.Country) AS country,
  toFloat(row.Latitude) AS lat,
  toFloat(row.Longitude) AS lon,
  point({latitude: toFloat(row.Latitude), longitude: toFloat(row.Longitude)}) AS geolocation
MERGE (a:Airport {iata: iata})
ON CREATE SET
  a.city = city,
  a.country = country,
  a.geolocation = geolocation
RETURN DISTINCT 'Complete';
"""

# Load Route Nodes

In [13]:
query_routes = """
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/Cryogenic117/graph-summit-demo/refs/heads/main/df_routes.csv" AS row
WITH
  trim(row.Departure) AS departure,
  trim(row.Arrival) AS arrival,
  toFloat(row.Distance_km) AS distance,
  toFloat(row.Duration_min) AS duration
MERGE (r:Route {departure: departure, arrival: arrival})
  ON CREATE SET r.distance = distance,
                r.duration = duration,
                r.id = randomUUID();
"""

# Load Airline Nodes

In [14]:
query_airlines = """
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/Cryogenic117/graph-summit-demo/refs/heads/main/df_routes.csv" AS row
WITH trim(row.Airline_IATA) AS airline
MERGE (a:Airline {iata: airline});
"""

# Load Airline Properties

In [15]:
query_airline_props = """
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/Cryogenic117/graph-summit-demo/refs/heads/main/airline_data.csv" AS row
WITH trim(row.IATA) AS iata,
     trim(row.Airline) AS airlineName,
     trim(row.`Country/Region`) AS region,
     trim(row.Comments) AS comments
MATCH (a:Airline {iata: iata})
SET a.name = CASE WHEN airlineName IS NOT NULL AND airlineName <> "" THEN airlineName ELSE a.name END,
    a.region = CASE WHEN region IS NOT NULL AND region <> "" THEN region ELSE a.region END,
    a.comments = CASE WHEN comments IS NOT NULL AND comments <> "" THEN comments ELSE a.comments END
"""

# Load All Relationships

In [16]:
query_relationships = """
LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/Cryogenic117/graph-summit-demo/refs/heads/main/df_routes.csv" AS row
WITH
  trim(row.Departure) AS departure,
  trim(row.Arrival) AS arrival,
  trim(row.Airline_IATA) AS airline
MATCH (dep:Airport {iata: departure})
MATCH (arr:Airport {iata: arrival})
MATCH (r:Route {departure: departure, arrival: arrival})
MERGE (dep)-[:HAS_ROUTE]->(r)
MERGE (r)-[:DESTINATION]->(arr)
WITH departure, arrival, airline, r
MATCH (a:Airline {iata: airline})
MERGE (a)-[:OPERATES]->(r);
"""

# Run Queries

In [17]:
queries = [
    ("Airport Constraint Creation", airport_constraint),
    ("Airline Constraint Creation", airline_constraint),
    ("Route Constraint Creation", route_constraint),
    ("Airport Nodes Creation", query_airports),
    ("Route Nodes Creation", query_routes),
    ("Airline Nodes Creation", query_airlines),
    ("Airline Properties Creation", query_airline_props),
    ("Relationships Creation", query_relationships)
]

progress_bar = widgets.IntProgress(value=0, min=0, max=len(queries))
status_label = widgets.Label(value="Starting...")
display(progress_bar, status_label)

driver = GraphDatabase.driver(URI, auth=AUTH)
with driver.session() as session:
    for i, (query_name, query) in enumerate(queries, start=1):
        status_label.value = f"Running {query_name}..."
        start_time = time.time()
        session.run(query)
        elapsed = time.time() - start_time
        status_label.value = f"Completed {query_name} in {elapsed:.2f} seconds."
        progress_bar.value = i
        time.sleep(1)
driver.close()

status_label.value = "All queries completed."

IntProgress(value=0, max=8)

Label(value='Starting...')